In [6]:
import pandas as pd
import numpy as np
import requests
import json
import os
import sqlalchemy
from sqlalchemy import create_engine
import time
from tqdm.notebook import tqdm
from io import StringIO
from google.cloud import storage

In [8]:
#setup redshift conncetion
REDSHIFT_ENDPOINT = 'juntian-cluster.cl4tv7kd0trb.us-east-1.redshift.amazonaws.com:5439/dev'
REDSHIFT_USER = 'awsuser'
REDSHIFT_PASS = 'arn:aws:secretsmanager:us-east-1:381492032483:secret:redshift!different-states-hospital-price-awsuser-FrEDZG'
REDSHIFT_PORT = '5439'
REDSHIFT_DB = 'different-states-hospital-price'

# Create the connection
connection = f"postgresql+psycopg2://{REDSHIFT_USER}:{REDSHIFT_PASS}@{REDSHIFT_ENDPOINT}:{REDSHIFT_PORT}/{REDSHIFT_DB}"
        
#Create engine
engine = create_engine(connection)

In [9]:
#load cpt_hcpcs.json into dataframe
# Set service account key
service_account_key_path = 'different-state-hospital-price-fd662d2f48c2.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key_path

# Specify JSON file path
bucket_path = 'gs://different-state-hospital-prices/cpt_hcpcs.json'

# Read JSON file into DataFrame
cpt_hcpcs_df = pd.read_json(bucket_path, lines=True)

print(cpt_hcpcs_df.info())
cpt_hcpcs_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287818 entries, 0 to 3287817
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code               object
 1   short_description  object
 2   long_description   object
dtypes: object(3)
memory usage: 75.3+ MB
None


,code,short_description,long_description
0,00000A,DVC REVASC 6X20MM 200CM,None
1,00001U,RBC DNA HEA 35 AG PLA,None
2,"00001U,1",RBC DNA HEA 35 AG PLA,None
3,00013,PT INDIVIDUAL GYM,None
4,0001A,HC ADM PFIZER SARSCOV2 30MCG/0.3ML 1ST,None


In [4]:
#cpt_hcpcs transformation
cpt_hcpcs_df = cpt_hcpcs_df.astype({
    'code': 'string',
    'short_description': 'string',
    'long_description': 'string'
})
cpt_hcpcs_df = cpt_hcpcs_df.fillna('None')

cpt_hcpcs_df = cpt_hcpcs_df.drop_duplicates()

print(cpt_hcpcs_df.info())
cpt_hcpcs_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287818 entries, 0 to 3287817
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code               string
 1   short_description  string
 2   long_description   string
dtypes: string(3)
memory usage: 75.3 MB
None


,code,short_description,long_description
0,00000A,DVC REVASC 6X20MM 200CM,None
1,00001U,RBC DNA HEA 35 AG PLA,None
2,"00001U,1",RBC DNA HEA 35 AG PLA,None
3,00013,PT INDIVIDUAL GYM,None
4,0001A,HC ADM PFIZER SARSCOV2 30MCG/0.3ML 1ST,None


In [5]:
#load hospitals.json into dataframe
bucket_path = 'gs://different-state-hospital-prices/hospitals.json'
hospitals_df = pd.read_json(bucket_path, lines=True)

print(hospitals_df.info())
hospitals_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   npi_number      1400 non-null   object 
 1   name            1400 non-null   object 
 2   url             1400 non-null   object 
 3   street_address  1351 non-null   object 
 4   city            1379 non-null   object 
 5   state           1377 non-null   object 
 6   zip_code        1357 non-null   object 
 7   publish_date    610 non-null    float64
dtypes: float64(1), object(7)
memory usage: 87.6+ KB
None


,npi_number,name,url,street_address,city,state,zip_code,publish_date
0,1003139775.0,HCA Virginia,https://hcavirginia.com/about/legal/pricing-tr...,901 E. Cary St Suite 210,Richmond,VA,None,1.609459e+12
1,1003260480,Brookwood Baptist Medical Center,https://www.brookwoodbaptisthealth.com/docs/gl...,2010 Brookwood Medical Center Dr.,Birmingham,AL,35209,NaN
2,1003281452,Henderson Hospital,https://uhsfilecdn.eskycity.net/ac/henderson-h...,1050 West Galleria Drive,Henderson,NV,89011,1.609459e+12
3,1003362997,CHI Health St. Elizabeth,https://www.chihealth.com/content/dam/chi-heal...,555 S. 70Th St.,Lincoln,NE,68510,1.609459e+12
4,1003389206,Merrill pioneer hospital,https://www.avera.org/app/files/public/79147/m...,"1100 S 10th Ave, Ste 100",Rock Rapids,IA,51246-2020,NaN


In [6]:
#hospitals transformation
#drop the uncessary column
hospitals_df.drop(['publish_date'],axis='columns')

#Unify npi_number
hospitals_df['npi_number'] = hospitals_df['npi_number'].astype(str)
hospitals_df['npi_number'] = hospitals_df['npi_number'].str.replace(r'\D', '', regex=True)
hospitals_df['npi_number'] = hospitals_df['npi_number'].apply(lambda x: x[:10] if len(x) >= 10 else None).astype('string')

#unify zip_code
hospitals_df['zip_code'] = hospitals_df['zip_code'].astype(str)
hospitals_df['zip_code'] = hospitals_df['zip_code'].str.replace('.0','')

# covert other types using astype
hospitals_df = hospitals_df.astype({
    'name': 'string',
    'url': 'string',
    'street_address':'string',
    'city':'string',
    'state':'string',
})

hospitals_df = hospitals_df.fillna('None')

hospitals_df = hospitals_df.drop_duplicates()
print(hospitals_df.info())
hospitals_df.head()

0   2021-01-01
1          NaT
2   2021-01-01
3   2021-01-01
4          NaT
Name: publish_date, dtype: datetime64[ns]
<class 'pandas.core.frame.DataFrame'>
Index: 1378 entries, 0 to 1399
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   npi_number            1378 non-null   string
 1   name                  1378 non-null   string
 2   url                   1378 non-null   string
 3   street_address        1378 non-null   string
 4   city                  1378 non-null   string
 5   state                 1378 non-null   string
 6   zip_code              1378 non-null   object
 7   publish_date          1378 non-null   string
 8   publish_date_year     1378 non-null   string
 9   publish_date_quarter  1378 non-null   string
 10  publish_date_month    1378 non-null   string
 11  publish_date_day      1378 non-null   string
dtypes: object(1), string(11)
memory usage: 140.0+ KB
None


,npi_number,name,url,street_address,city,state,zip_code,publish_date,publish_date_year,publish_date_quarter,publish_date_month,publish_date_day
0,1003139775,HCA Virginia,https://hcavirginia.com/about/legal/pricing-tr...,901 E. Cary St Suite 210,Richmond,VA,None,2021-01-01,2021,1,1,1
1,1003260480,Brookwood Baptist Medical Center,https://www.brookwoodbaptisthealth.com/docs/gl...,2010 Brookwood Medical Center Dr.,Birmingham,AL,35209,None,None,None,None,None
2,1003281452,Henderson Hospital,https://uhsfilecdn.eskycity.net/ac/henderson-h...,1050 West Galleria Drive,Henderson,NV,89011,2021-01-01,2021,1,1,1
3,1003362997,CHI Health St. Elizabeth,https://www.chihealth.com/content/dam/chi-heal...,555 S. 70Th St.,Lincoln,NE,68510,2021-01-01,2021,1,1,1
4,1003389206,Merrill pioneer hospital,https://www.avera.org/app/files/public/79147/m...,"1100 S 10th Ave, Ste 100",Rock Rapids,IA,51246-2020,None,None,None,None,None


In [ ]:
#load prices.json into dataframe
bucket_path = 'gs://different-state-hospital-prices/prices.json'

prices_df = pd.DataFrame()

# Create a JSON reader
json_reader = pd.read_json(bucket_path, lines=True, chunksize=2000)

# Process each chunk
for chunk in json_reader:
    # Clean 'npi_number' column
    chunk['npi_number'] = chunk['npi_number'].astype(str)
    chunk['npi_number'] = chunk['npi_number'].str.replace(r'\D', '', regex=True)
    chunk['npi_number'] = chunk['npi_number'].apply(lambda x: x[:10] if len(x) >= 10 else None).astype('string')

    # change dtype
    chunk = chunk.astype({
        'code': 'string',
        'payer': 'string'
    })
    
    # Append to dataframe
    prices_df = pd.concat([prices_df, chunk], ignore_index=True)
    
prices_df = prices_df.fillna('none')

prices_df = prices_df.drop_duplicates()

print(prices_df.head())
print(prices_df.info())

In [ ]:
#create new dataframe
